# 기본 설정

## 라이브러리 설치 및 api key 입력

In [7]:
# 라이브러리 설치 (필요 시)

import os, sys
import sqlite3
import pandas as pd
from textwrap import dedent
from openai import OpenAI

In [ ]:
client = OpenAI(api_key=api_key)
print("✅ OpenAI API Key Loaded (prefix):", api_key[:8], "***")

✅ OpenAI API Key Loaded (prefix): sk-proj- ***


## DB 파일 및 테이블 생성

In [ ]:
csv_file_path1 = 'T1_도서관기본정보.csv'
csv_file_path2 = 'T2_장서정보.csv'
csv_file_path3 = 'T3_시설현황.csv'
csv_file_path4 = 'T4_이용자정보_지역포함.csv'
csv_file_path5 = 'T5_취약계층_어린이서비스.csv'
csv_file_path6 = 'T6_지역인구.csv'

DB_PATH = 'CatDewey.db'

df1 = pd.read_csv(csv_file_path1, encoding='utf-8')
df2 = pd.read_csv(csv_file_path2, encoding='utf-8')
df3 = pd.read_csv(csv_file_path3, encoding='utf-8')
df4 = pd.read_csv(csv_file_path4, encoding='utf-8')
df5 = pd.read_csv(csv_file_path5, encoding='utf-8')
df6 = pd.read_csv(csv_file_path6, encoding='utf-8')

conn = sqlite3.connect(DB_PATH)
cur = conn.cursor()

df1.to_sql('base_info', conn, if_exists='replace', index=False)
df2.to_sql('holding', conn, if_exists='replace', index=False)
df3.to_sql('fac', conn, if_exists='replace', index=False)
df4.to_sql('user', conn, if_exists='replace', index=False)
df5.to_sql('sevice', conn, if_exists='replace', index=False)
df6.to_sql('pop', conn, if_exists='replace', index=False)

conn.commit()

UnicodeDecodeError: 'euc_kr' codec can't decode byte 0x8f in position 22: illegal multibyte sequence

## 데이터베이스 확인

In [ ]:
def get_tables(conn):
    q = "SELECT name FROM sqlite_master WHERE type='table' AND name NOT LIKE 'sqlite_%';"
    return [row[0] for row in conn.execute(q)]

def get_table_columns(conn, table_name):
    q = f"PRAGMA table_info({table_name});"
    df = pd.read_sql_query(q, conn)
    return df

tables = get_tables(conn)
print('테이블 목록:', tables)

schema_docs = []
for t in tables:
    cols = get_table_columns(conn, t)
    display(cols)
    schema_docs.append({
        'table': t,
        'columns_df': cols
    })

# SQL 질의 예시

In [ ]:
def run_sql(query: str, params=None):
    """도움 함수: SQL을 실행하고 DataFrame으로 결과를 보여줍니다."""
    if params is None:
        params = []
    result_df = pd.read_sql_query(query, conn, params=params)
    display(result_df)

print("✅ run_sql 함수 준비 완료")

In [ ]:
# 예시 1: 모든 상품 조회
print("[모든 상품 목록]")
run_sql("SELECT * FROM products")

# 예시 2: 최근 30일 주문 내역
print("[최근 30일 주문]")
recent_date = (today - timedelta(days=30)).strftime("%Y-%m-%d")
run_sql(
    """
    SELECT o.id AS order_id, u.name AS user_name, o.order_date, o.total_amount
    FROM orders o
    JOIN users u ON o.user_id = u.id
    WHERE o.order_date >= ?
    ORDER BY o.order_date;
    """,
    [recent_date],
)